In [1]:
import torch
import seaborn as sea
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from game_log import load_current_line, GameLog
plt.rcParams['figure.figsize'] = [15, 8]
pd.set_option("display.precision", 3)

In [2]:
pd.set_option('display.max_rows', 10)
pd.set_option('display.min_rows', 10)
torch.set_printoptions(sci_mode=False)

In [3]:
opp_stats = pd.read_csv('opponent_stats.csv',index_col=0)
schedule = pd.read_csv('schedule.csv',index_col=0)
KM_vals = pd.read_csv('KM_vals.csv',index_col=0)

today = dt.datetime.today()
year = today.year
month = today.month
day = today.day
today_str = f'{year}_{month}_{day}'


pp_path = f'Lines/pp/pp_{today_str}.csv'
unabated_path = f'Lines/unabated/unabated_{today_str}.csv'

pp_lines = load_current_line(pp_path,'prop_id','time')
unabated = load_current_line(unabated_path,'prop_id','time')

In [4]:
pdata = pd.read_csv('pdata.csv')
pdata['season'] = pdata['date'].apply(lambda x: x.split('-')[0])
data = pd.read_csv('data_2024.csv')
data23 = pd.read_csv('data_2023.csv')
data = pd.concat((pdata,data,data23))
data= data.sort_values(by=['player','date']).reset_index(drop=True)
data.sample(3)

,player,G,date,series,team,H/A,opp,G#,W/L,GS,...,TOV,PF,PTS,GmSc,+/-,pos,KM,season,age,Opp
1353,Anthony Black,2,2023-10-31,NaN,ORL,0,NaN,NaN,-16,0,...,0,1,0,0.3,-1.0,PG,15,2024,19-284,LAC
29649,Robin Lopez,2,2020-08-20,EC1,MIL,1,ORL,2.0,15,0,...,0,2,2,0.9,0.0,C,15,2020,NaN,NaN
14770,James Harden,15,2022-12-19,NaN,PHI,1,NaN,NaN,3,1,...,5,2,14,9.1,14.0,PG,15,2023,33-115,TOR


In [5]:
gl = GameLog(data)

In [48]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
def pt_corr(player_a_name, player_b_name, team,stat1='PTS',stat2='PTS'):
    team_games = data.loc[data.team==team]
    # extract two columns of interest (i.e. points scored by the two players)
    player_a_dates = set(team_games[team_games['player'] == player_a_name]['date'])
    player_b_dates = set(team_games[team_games['player'] == player_b_name]['date'])
    common_dates = player_a_dates.intersection(player_b_dates)
    stats = team_games[team_games['date'].isin(common_dates)]
    
    player_a_points = stats[stats['player'] == player_a_name][stat1].reset_index(drop=True)
    player_b_points = stats[stats['player'] == player_b_name][stat2].reset_index(drop=True)
    sample = len(player_a_points)
    # calculate the correlation coefficient between the two columns
    corr_coeff = player_a_points.corr(player_b_points)
    # return the correlation coefficient
    return corr_coeff,sample
pt_corr('Stephen Curry','Klay Thompson','GSW')

(0.035121793751097447, 189)

In [7]:
team_list = data.team.unique()

In [25]:
score_avg = data.groupby('player')['PTS'].mean().reset_index().sort_values(by='PTS',ascending=False)
top_scorers = score_avg.player.unique()[0:125]

In [27]:
team2players = {}
for team in team_list:
    team_games = data.loc[data.team==team]
    players = team_games.loc[team_games.GS==1].player.unique()
    team2players[team]=players

In [59]:
stats = ['PTS','AST','TRB']
def get_correlations(team_list,player_list,stat1_list,stat2_list):
    correlate = []
    for team in team_list:
        for p in team2players[team]:
            if p in player_list:
                for p2 in team2players[team]:
                    if (p != p2) and (p2 in player_list):
                        for stat1 in stat1_list:
                            for stat2 in stat2_list:
                                corr,sample = pt_corr(p,p2,team,stat1,stat2)
                                correlate.append([team,sample,p,p2,stat1,corr,stat2])
                                corr,sample = pt_corr(p2,p,team,stat2,stat1)
                                correlate.append([team,sample,p2,p,stat2,corr,stat2])
    all_corr = pd.DataFrame(correlate,columns=['Team','Sample','P1','P2','Stat1','Corr','stat2']).sort_values(by='Corr').dropna()
    all_corr = all_corr.loc[all_corr['Sample']>10]
    return all_corr
all_corr = get_correlations(team_list,top_scorers,stats,stats)

c:\Users\Shared.Westover\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\Shared.Westover\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\Shared.Westover\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\Shared.Westover\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2705: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
c:\Users\Shared.Westover\AppData\Local\anaconda3\Lib\site-packages\numpy\lib\function_base.py:2846: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar, dtype=dtype)
c:\Users\Shared.Westover\AppData\Local\anaconda3\Lib\site-packages\nump

In [66]:
tonight = ['TOR','PHI','DET','NOP','ORL','UTA','PHO','SAS']
all_corr.reset_index(drop=True).drop_duplicates(inplace=True)
all_corr.loc[(all_corr.Sample > 20) & (all_corr.Team.isin(['PHI']))].tail(10)

,Team,Sample,P1,P2,Stat1,Corr,stat2
11613,PHI,77,Joel Embiid,Tyrese Maxey,AST,0.178,AST
11328,PHI,77,Joel Embiid,Tyrese Maxey,AST,0.178,PTS
11254,PHI,64,Joel Embiid,James Harden,PTS,0.199,TRB
11065,PHI,64,Joel Embiid,James Harden,PTS,0.199,PTS
11255,PHI,64,James Harden,Joel Embiid,TRB,0.199,TRB
11064,PHI,64,James Harden,Joel Embiid,TRB,0.199,PTS
11252,PHI,64,Joel Embiid,James Harden,PTS,0.464,AST
11253,PHI,64,James Harden,Joel Embiid,AST,0.464,AST
11058,PHI,64,James Harden,Joel Embiid,AST,0.464,PTS
11059,PHI,64,Joel Embiid,James Harden,PTS,0.464,PTS


In [67]:
all_corr.loc[all_corr.P1== 'Joel Embiid'].head(10)

,Team,Sample,P1,P2,Stat1,Corr,stat2
11169,PHI,11,Joel Embiid,Jimmy Butler,AST,-0.500,AST
11276,PHI,11,Joel Embiid,Jimmy Butler,AST,-0.500,AST
11282,PHI,11,Joel Embiid,Jimmy Butler,TRB,-0.378,AST
11171,PHI,11,Joel Embiid,Jimmy Butler,TRB,-0.378,TRB
11161,PHI,11,Joel Embiid,Jimmy Butler,PTS,-0.374,PTS
11268,PHI,11,Joel Embiid,Jimmy Butler,PTS,-0.374,PTS
11173,PHI,11,Joel Embiid,Jimmy Butler,PTS,-0.253,PTS
11272,PHI,11,Joel Embiid,Jimmy Butler,PTS,-0.253,TRB
11177,PHI,11,Joel Embiid,Jimmy Butler,TRB,-0.243,TRB
11284,PHI,11,Joel Embiid,Jimmy Butler,TRB,-0.243,TRB


In [ ]:
all_corr.to_csv('correlation.csv',index=0)